# The Unique Properties of Qubits

*This workshop is a companion to 1.1 - The Unique Properties of Qubits.*

We'll now go back to the first program, of Workshop 1.1 and add some additional features.

A big difference between qubits and normal bits is that there is only one way to get an output from a normal bit: You just look at it, and see if it is `0` or `1`. For a qubit, however, there are multiple ways you can choose to do it. One is the method applied when you use a simple `measure` operation. This is called a z measurement.

```python
# z measurement of qubit j
qc.measure(j,j)
```

Another form of measurement, known as an x measurement, cannot be performed quite as directly in Qiskit. Instead, it needs two lines.

```python
# x measurement of qubit j
qc.h(j)
qc.measure(j,j)
```

In the following we expand upon the `meas` variable introduced in the last workshop. There it held just a single circuit. Here it will hold four: a single qubit measurement for each qubit, and for each basis. It is constructed as follows.

```python
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
```

For example, `meas[0]['X']` contains the circuit that measures qubit 0 in the x basis.

```python
meas[0]['X'] = QuantumCircuit(2,2)
meas[0]['X'].h(j)
meas[0]['Z'].measure(j,j)
```

The reason why these single qubit measurements are defined in two qubit circuits is simply so that they can be easily added together. For example, the circuit with a z measurement on qubit 0 and an x measurement on qubit 1 corresponds to the circuit `meas[0]['Z']+meas[1]['X']`.

The choice as to which measurement type to use will be controlled by the variable `basis`. The name is chosen because the type of measurement is also known as the measurement basis. We will set `basis[0]` will be `'Z'` or `'X'` depending on whether qubit 0 should be measured in basis z or x, respectively, and similarly for `basis[1]` and qubit 1. This means we can access the circuit for the measurement of qubit `j` with `meas[j][basis[j]]`.

To make it obvious which basis is being used, we will display the results in different places. The output of a z measurement of qubit 1 will be displayed on pixel (1,2), as before. The output of an x measurement will instead be displayed below this at (1,4). Both will have square of dim pixels around them. And both will also be set to dim when their basis is not in use.

To choose which basis to use for each qubit, we'll use the buttons X and O. The X button will toggle between the two bases for qubit 1, and the O button will do the same for qubit 0.

In [ ]:
%matplotlib notebook

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = execute(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

In the program above, it is as if each qubit is hiding in a box. If we peek into the hole at the top, we see its z basis information. For the hole at the bottom, we see the x basis information.

Actually, we are seeing many qubits, since a new pair is generated and simulated every frame. But since each pair is created in the same state, it gives us a glimpse into the life of a qubit.

To get even more of an insight, let's add the ability to change the state that is being simulated. We'll do this by restoring some of the lines we had before, which added `x` gates into `qc` when ▲ and ◀︎ are pressed. Similarly, we can add an `h` to `qc` when ▼ and ► are pressed.

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys&pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys&pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = execute(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

The program below introduces two slight changes to the one above.

The first is to change the initial state. Instead of the normal initial state, which always outputs `00` for a z measurement of both qubits, we use the state created with

```python
qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
qc.h(0)
qc.cx(0,1)
```

As a simple way to think of this, we can note that the `h(0)` has the effect of creating a state for which the z measurement of qubit 1 gives a random outcome. Qubit 1 remains in a state for which the outcome `0` is certain for a z measurement.

As we have seen before, the effect of the `cx` gate on the z basis is to calculate an XOR. The XOR of the random value from qubit 0 with the `0` from qubit 1 is simply the random value from qubit 0, since the XOR of any bit value with `0` returns the same bit value. The `cx(0,1)` therefore has the effect of copying the random value over to qubit 2.

Given this behaviour, the state we have preapred is one for which the two qubits always give the same random outcome for a z measurement. The only possible outcomes for a z measurement of both qubits are therefore `'00'` and `'11'`.

To show this more clearly, we add some lines to our program that light up a few extra pixels whenever the outcome is indeed `'00'` or `'11'`. Specifically, a pair at the top will be lit up for `'00'`, and a pair at the bottom for `'11'`.

Try it out, and also take a look at the behaviour for an x measurement of both.

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
qc.h(0)
qc.cx(0,1)

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys==pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys==pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys==pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys==pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys==pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys==pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = execute(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)
        
    # flag up the '11' output by lighting up pixels in the bottom-center
    if m[0]=='11':
        screen.pixel(3,6,3)
        screen.pixel(4,6,3)
    else:
        screen.pixel(3,6,0)
        screen.pixel(4,6,0)
    # same with '00' and the top center
    if m[0]=='00':
        screen.pixel(3,0,3)
        screen.pixel(4,0,3)
    else:
        screen.pixel(3,0,0)
        screen.pixel(4,0,0)  

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

For an x measurement of one qubit and a z measurement of the other, the two outputs are just as likely to agree as disagree. We will find no correlation between them.

For an x measurement of both qubits, however, we find a new correlation. Just as for a z measurement of both, the outputs of each qubit will always agree.

This suggests that there is another interpretation of the action of the `cx` gate, in terms of the x basis. For an x basis measurement it is qubit 1 that starts off in the state that gives a random outcome. The action of the `h(0)` makes qubit 0 the one that is sure to output a `0`. The effect of the `cx` interpreted in terms of these x basis outcomes is therefore also consistent with the action of an XOR. However, in this case, it is the 'control' qubit on which the outcome is written, rather than the 'target'.

Playing again with the program above, try to make a state for which the z measurements always disagree. Try also the same for the x measurements. You can also try to make a state for which an x measurement of one qubit and a z of the other will always agree or disagree.

---

Now we will change the program, by just changing the initial state. Specifically, we will create a state for which z measurements of both qubits will never output `'00'`, but will output the other three possibilities with equal probability.

This can be easily done if we know the statevector for this state. These vectors are a list with one entry for each possible output. For two qubits, this means a list of four elements, for `'00'`, `'01'`, `'10'` and `'11'`, respectively.

These elements are the square roots of the probabilities for each outcome (when both qubits are measured in the z basis). Since the non-zero probabilties should be $1/3$, the corresponding element should be $0.57735$. So the state vector will be

```python
[0,0.57735,0.57735,0.57735]
```

Using the `initialize` method of a `QuantumCircuit` object, we can then set this state up

```python
qc.initialize([0,0.57735,0.57735,0.57735])
```

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
qc.initialize([0,0.57735,0.57735,0.57735])

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys&pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys&pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = execute(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)
        
    # flag up the '11' output by lighting up pixels in the bottom-center
    if m[0]=='11':
        screen.pixel(3,6,3)
        screen.pixel(4,6,3)
    else:
        screen.pixel(3,6,0)
        screen.pixel(4,6,0)
    # same with '00' and the top center
    if m[0]=='00':
        screen.pixel(3,0,3)
        screen.pixel(4,0,3)
    else:
        screen.pixel(3,0,0)
        screen.pixel(4,0,0)  

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

As you should see, this state never outputs `'00'` when both qubits are measured in the z basis. However, this state happens to have other interesting properties too: For a z measurement of one qubit and and x measurement of the other, it never outputs `'11'`. Putting this another way, whenever an x measurement of one qubit outputs `1`, the other must output `0`.

Can we use this information to deduce something about the results for an x measurement of each? To do this we need to make some assumptions. Some of these are assumptions that we might hardly realize we are making.

For example, we are used to random results being due to imprecision of our own knowledge. A coin flip, for example, seems random only because don't know precisely where on the coin our thumb strikes, or with what force, or the pressure and movement of the air through which the coin travels, and so on. If we did know these things, we could predict the results with certainty. The same is true for the internal state of random number generators in our computers. It is therefore natural to assume that qubits similarly possess some 'hidden variables', which we could use to predict results if only we knew them. This would mean that, whenever we prepare a qubit in some state, we are actually randomly generating some set of hidden variables in a way that depends on that state.

Also, we typically assume that objects are independent from our observations of them. Even more so, we assume that one object is independent of how we observe another independent object. Combined with our our assumption of hidden variables, we would assume that the results for each qubit can be deduced using only the knowledge of how that qubit is being measured, along with the values of its hidden variables.

Now, with these assumptions, let us consider whether it is possible to get an outcome of `'11'` for an x measurement of both qubits.

We have already seen that an outcome of `1` for an x measurement of one qubit implies that a z measurement for the other must output a `0`. That would imply that whatever value for the hidden variables of the qubits that cause the outcome of `'11'` for x measurements, would cause an outcome of `'00'` for z measurements. We have seen that the latter never occurs, and so can conclude that the particular state we initialize never has hidden variables that would allow such an outcome. This means that an outcome of `'11'` for x measurements cannot occur either.

Try it out, and see how wrong we are!

The truth is that the outcome of `'11'` for x measurements is indeed possible for this state. Our assumptions must therefore be incorrect. Specifically, we can either give up the idea that the randomness is dependent on hidden variables, or the idea that the results we get from each qubit is independent of how we choose to measure the other.

After many experiments over the last few decades, we have concluded that it is the assumption of hidden variables that must be abandoned. The randomness of qubits is therefore truly random. A qubit certain to output a `0` for a x measurement, for example, has truly no idea of the outcome it would give for an x measurement. Any certainty we have for the outcome of one measurement type implies uncertainty for the output of the other.

This is exactly why the logic we used above did not hold. We took a case where we knew the outcome of both qubits for an x measurement, and used that to infer what would have happened if one of the qubits had a z measurement. This meant we knew the outcomes of both measurement types with certainty, and so we were wrong.

The behaviour we see here shows how qubits are intrinsically different from any variable in normal programming. If we created two ways of extracting a bit value from an integer, or a list, or some other standard object, we could never prepare two such objects that were correlated in the same way as these qubits. Indeed, when we simulate a group of qubits, we cannot do so by creating a separate object to handle to simulation of each. If we could, these could simply be used as our hidden variables when describing real qubits. Instead, we need to create a single object to handle the entire simulation, which must be able to account for every possible combination of measurements. It is precisely because of this reason that quantum computers become exponentially hard to simulate as the number of qubits increases. And that is why we know that they have the potential to do things that no normal computer can reproduce. We just need to harness the counterintuive effects, such as the one we've seen here, to solve our computational problems.